<a href="https://colab.research.google.com/github/JeyTee1988/LLM-Schulung/blob/main/llm_schulung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Wichtig**: Wenn Du dieses Notebook in Google Colab ausführst, wähle zuerst unter Laufzeit > Laufzeittyp ändern eine GPU aus

Zunächst installieren wir die benötigte Library Huggingface Transformer

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.4 MB/s eta 0:00:00


# Klassifizierung von Text

Wir laden eine Pipeline mit einem vortrainierten Sentiment-Modell. Das Modell kann explizit angegeben werden, ansonsten wird ein Standard-Modell genommen.

In [2]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Wir können das Modell nutzen, um die Stimmung eines Textes zu bestimmen. Hierbei wird angegeben, ob der Text positiv oder negativ ist und wie sicher das Modell ist bei der Bewertung.

In [3]:
classifier(["We are happy to have you participate in our modest LLM workshop!", "Sorry that we had to cap the participant limit to 20"])

[{'label': 'POSITIVE', 'score': 0.9998558759689331},
 {'label': 'NEGATIVE', 'score': 0.9991243481636047}]

Dieses Modell wurde nur mit englischen Texten und scheinbar nicht mit Emoji trainiert:

In [4]:
classifier(["Was ein schöner Tag für eine Schulung", "🤗", "😠"])

[{'label': 'NEGATIVE', 'score': 0.9137821793556213},
 {'label': 'NEGATIVE', 'score': 0.6970565319061279},
 {'label': 'NEGATIVE', 'score': 0.6970565319061279}]

# Textgenerierung

Wir laden das vortrainierte distilgpt2 Modell und nutzen es für Textgenerierung

In [5]:
from transformers import pipeline

pipe = pipeline("text-generation", model="distilgpt2")

Wir können das Modell nutzen, um Texte zu vervollständigen.

In [91]:
from transformers import set_seed
set_seed(42)
pipe("Hello, I’m a language model", max_length=20, num_return_sequences=5)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello The'},
 {'generated_text': 'HelloR'},
 {'generated_text': 'Hello�'},
 {'generated_text': 'HelloF'},
 {'generated_text': 'HelloIn'}]

# Tokenizer

Ein Tokenizer in einem Language Learning Model (LLM) dient dazu, Text in kleinere Einheiten, sogenannte Tokens, zu zerlegen. Diese Tokens können einzelne Wörter, Phrasen oder sogar einzelne Buchstaben sein.

Der Tokenizer ist wichtig, weil ein LLM wie ein neuronales Netzwerk arbeitet, das auf Zahlen, nicht auf Wörtern, basiert. Es kann nicht direkt Text verstehen oder interpretieren. Stattdessen muss es den Text in eine numerische Form umwandeln, die es verarbeiten kann.

Dieser Prozess des Zerlegens und Umwandeln wird durch den Tokenizer durchgeführt. Er nimmt den ursprünglichen Text, teilt ihn in Tokens und wandelt diese dann in Zahlen um, die das Modell verarbeiten kann.

https://platform.openai.com/tokenizer


In [96]:
tokenizer = pipe.tokenizer
model = pipe.model

text = "Hello, I’m a language model"
tokenizer.tokenize(text)

['Hello', ',', 'ĠI', 'âĢ', 'Ļ', 'm', 'Ġa', 'Ġlanguage', 'Ġmodel']

In [97]:
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input.input_ids

tensor([[15496,    11,   314,   447,   247,    76,   257,  3303,  2746]])

Der Tokenizer enthält ein Dictionary, welches jedem Token eine Zahl zuweist

In [98]:
vocab = tokenizer.get_vocab()
first_20_items = {k: vocab[k] for k in list(vocab.keys())[:20]}
first_20_items

{'Ġdefiance': 35287,
 'Ġfreed': 13459,
 'opes': 13920,
 'Ġexpensive': 5789,
 'estation': 27364,
 'ĠSoft': 8297,
 'Econom': 28489,
 'Ġloves': 10408,
 'Ġarresting': 33936,
 'Ġturbo': 29292,
 'RN': 42336,
 'ĠTDs': 42873,
 'Ġproduct': 1720,
 'ĠBirth': 17647,
 'ĠEmpty': 33523,
 'Ġadhesive': 43608,
 'ins': 1040,
 'irmed': 15491,
 'an': 272,
 'Ġbigger': 5749}

Es gibt insgesamt 50257 Tokens in diesem Tokenizer.

In [83]:
len(vocab)

50257

In [99]:
output = model(**encoded_input)
token_logits = output.logits

# Get the predicted token ids
predicted_token_ids = token_logits.argmax(dim=-1)
predicted_token_ids

tensor([[ 383,  290,  447,  247,   76,  407, 4336, 5887,   13]])

In [101]:
# Decode the tokens to text
predicted_text = [tokenizer.decode(t) for t in predicted_token_ids]

for index, t in enumerate(predicted_token_ids[0]):
  input_token = tokenizer.tokenize(text)[index]
  predicted_next_token = tokenizer.decode(t)
  print(input_token + " => " + predicted_next_token)


Hello =>  The
, =>  and
ĠI => �
âĢ => �
Ļ => m
m =>  not
Ġa =>  fan
Ġlanguage =>  expert
Ġmodel => .


# Weitere Arten von Tasks
Auf folgender Seite findet ihr weitere mögliche Anwendungsfälle
https://huggingface.co/docs/transformers/main/en/quicktour#pipeline

Probiert gerne welche aus 🤗